In [1]:
%pip install -q langchain langchain-community

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\rag\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import os
from langchain.document_loaders import PyMuPDFLoader
# from langchain.vectorstores import SimpleVectorStore
from langchain.embeddings import OpenAIEmbeddings

In [2]:
def load_pdfs_from_folder(folder_path):
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
    documents = []
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(folder_path, pdf_file)
        loader = PyMuPDFLoader(pdf_path)
        docs = loader.load()
        documents.extend(docs)
    
    return documents

In [3]:
folder_path = r"vibration"
docs = load_pdfs_from_folder(folder_path)


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
folder_path = r"vibration"
docs = load_pdfs_from_folder(folder_path)

chunked_docs = splitter.split_documents(docs)

In [6]:
len(chunked_docs)

130

In [7]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
 

db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))

d:\rag\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
d:\rag\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\rag\venv\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
 

In [7]:
import faiss

# Retrieve the underlying FAISS index
faiss_index = db.index

# Save the FAISS index to a file
faiss.write_index(faiss_index, "faiss_index.index")

print("FAISS index saved to faiss_index.index")

FAISS index saved to faiss_index.index


In [8]:
import pickle

with open("chunked_docs.pkl", "wb") as f:
    pickle.dump(chunked_docs, f)

NameError: name 'chunked_docs' is not defined

In [10]:
import faiss
import pickle
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Load the FAISS index from the file
faiss_index = faiss.read_index("faiss_index.index")

# Assuming you need to reconstruct the FAISS vector store
# You need to load the chunked_docs separately
with open("chunked_docs.pkl", "rb") as f:
    chunked_docs = pickle.load(f)

# Create the FAISS vector store with the loaded index and documents
db_loaded = FAISS(index=faiss_index, embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))

print("FAISS index and documents loaded successfully.")


EOFError: Ran out of input

In [27]:
def retrieve_from_vector_store(vector_store, query, k=5):
    # Perform the retrieval operation
    results = vector_store.similarity_search(query, k=k)
    return results

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
# os.environ['CURL_CA_BUNDLE'] = r"C:\Users\25025131\Cisco_Umbrella_Root_CA.cer"
os.environ['CURL_CA_BUNDLE'] = ''
os.environ['REQUESTS_CA_BUNDLE'] = ''

model_name = "HuggingFaceH4/zephyr-7b-beta"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
# )

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 8/8 [00:31<00:00,  3.92s/it]


In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

ImportError: Using `bitsandbytes` 4-bit quantization requires Accelerate: `pip install accelerate`

In [10]:
# Specify directories to save the model and tokenizer
model_save_path = "./saved_model"
tokenizer_save_path = "./saved_tokenizer"

# Save the model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model saved to {model_save_path}")
print(f"Tokenizer saved to {tokenizer_save_path}")

Model saved to ./saved_model
Tokenizer saved to ./saved_tokenizer


In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "HuggingFaceH4/zephyr-7b-beta"

# Paths where the model and tokenizer were saved
model_save_path = "./saved_model"
tokenizer_save_path = "./saved_tokenizer"

# Load the model and tokenizer from the saved paths
modelsave = AutoModelForCausalLM.from_pretrained(model_save_path)
tokenizersave = AutoTokenizer.from_pretrained(tokenizer_save_path)

Loading checkpoint shards: 100%|██████████| 6/6 [00:07<00:00,  1.23s/it]


In [9]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
    device='cuda'
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

d:\rag\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [11]:
from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

In [12]:
question = "What is Spring?"
llm_chain.invoke({"context": "", "question": question})

'\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n\n\n</s>\n<|user|>\nWhat is Spring?\n</s>\n<|assistant|>\n\n  Spring refers to the season that comes after winter and before summer, characterized by warmer temperatures, longer days, and new growth in nature. In a broader sense, Spring also refers to a software development framework called Spring Framework, which provides a comprehensive programming and configuration model for developing Java applications. This framework aims to simplify the process of building complex enterprise-level applications by providing features such as dependency injection, aspect-oriented programming (AOP), and support for data access technologies like JDBC and Hibernate.'

In [13]:
rag_chain.invoke(question)

'\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n[Document(metadata={\'source\': \'vibration\\\\Vibration Theory.pdf\', \'file_path\': \'vibration\\\\Vibration Theory.pdf\', \'page\': 0, \'total_pages\': 32, \'format\': \'PDF 1.4\', \'title\': \'8434_Harris_02_b.qxd\', \'author\': \'Prepress 1\', \'subject\': \'\', \'keywords\': \'\', \'creator\': \'QuarkXPressª 4.11: PSPrinter 8.3.1\', \'producer\': \'Acrobat Distiller 4.05 for Macintosh\', \'creationDate\': \'D:20010920123819Z\', \'modDate\': "D:20050202111026-05\'00\'", \'trapped\': \'\'}, page_content=\'to the force acting along its length:\\nF = k(x −u)\\n(2.1)\\nThe ideal spring is considered to have no mass;\\nthus, the force acting on one end is equal and\\n2.1\\nFIGURE 2.1\\nLinear spring.\'), Document(metadata={\'source\': \'vibration\\\\Vibration Theory.pdf\', \'file_path\': \'vibration\\\\Vibration Theory.pdf\', \'page\': 0, \'total_pages\': 32, \'format\': \'PDF 1.4\', \'tit

In [14]:
rag_chain.invoke("What is the rotational quantity corresponding to mass?")

'\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n[Document(metadata={\'source\': \'vibration\\\\Vibration Theory.pdf\', \'file_path\': \'vibration\\\\Vibration Theory.pdf\', \'page\': 2, \'total_pages\': 32, \'format\': \'PDF 1.4\', \'title\': \'8434_Harris_02_b.qxd\', \'author\': \'Prepress 1\', \'subject\': \'\', \'keywords\': \'\', \'creator\': \'QuarkXPressª 4.11: PSPrinter 8.3.1\', \'producer\': \'Acrobat Distiller 4.05 for Macintosh\', \'creationDate\': \'D:20010920123819Z\', \'modDate\': "D:20050202111026-05\'00\'", \'trapped\': \'\'}, page_content=\'Inasmuch as the mathematical equations for a rotational system can be written by\\nanalogy from the equations for a translational system, only the latter are discussed in\\ndetail.Whenever translational systems are discussed,it is understood that correspond-\\ning equations apply to the analogous rotational system, as indicated in Table 2.1.\\nSINGLE DEGREE-OF-FREEDOM SYSTEM\\nThe sim

In [3]:
import torch
torch.__version__

'2.4.0+cu118'

In [4]:
torch.cuda.is_available()

True